In [ ]:
import os

package_path = '/home/meganorm-smkia/Code/MEGaNorm/'
os.chdir(package_path)

from utils.parallel import submit_jobs, check_jobs_status, collect_results
from datasets.camcan import load_camcan_data
from utils.nm import hbr_data_split
from plots.plots import plot_nm_range2

import pcntoolkit as ptk

In [ ]:
### Paths and configs

username = 'meganorm-smkia'

mainParallel_path = os.path.join(package_path, 'src', 'mainParallel.py')

data_path = '/project/meganorm/Data/camcan/CamCAN/cc700/meg/pipeline/release005/BIDSsep/derivatives_rest/aa/AA_movecomp_transdef/aamod_meg_maxfilt_00003/'
target_dir = '/home/meganorm-smkia/Data/CamCAN/Features/'

log_path = '/home/meganorm-smkia/temp/log/'
temp_path = '/home/meganorm-smkia/temp/tmp/'

processing_dir = '/home/meganorm-smkia/Data/CamCAN/NM/'

job_configs = {'log_path':log_path, 'module':'mne', 'time':'1:00:00', 'memory':'20GB', 
                'partition':'normal', 'core':1, 'node':1, 'batch_file_name':'batch_job'}

if not os.path.isdir(log_path):
    os.makedirs(log_path)
    
if not os.path.isdir(processing_dir):
    os.makedirs(processing_dir)


In [ ]:
### Parallel feature extraction

subjects = os.listdir(data_path)
 
# Running Jobs
start_time = submit_jobs(mainParallel_path, target_dir, data_path, subjects, 
                temp_path, job_configs=job_configs)
# Checking jobs
failed_job_names= check_jobs_status(username, start_time)

while len(failed_job_names)>0:
    # Re-running Jobs
    start_time = submit_jobs(mainParallel_path, target_dir, data_path, failed_job_names, 
                temp_path, job_configs=job_configs)
    # Checking jobs
    failed_job_names= check_jobs_status(username, start_time)

collect_results(target_dir, subjects, temp_path, file_name='camcan_features')

In [ ]:
### Data preparation for Normative Modeling

camcan_cov_path = '/project/meganorm/Data/camcan/CamCAN/cc700/participants.tsv'
camcan_feature_path = '/home/meganorm-smkia/Data/CamCAN/Features/camcan_features.csv'

camcan_data = load_camcan_data(camcan_feature_path, camcan_cov_path)

biomarker_num = hbr_data_split(camcan_data, processing_dir, drop_nans=True, batch_effects=['gender','site'])


In [ ]:
### Setting up NM configs

python_path = '/project/meganorm/Software/Miniconda3/envs/mne/bin/python' 

hbr_configs = {'hetero_SHASH_bspline':{'model_type':'bspline', 'likelihood':'SHASHb', 'linear_sigma':'True',
                                   'random_slope_mu':'True', 'linear_epsilon':'True', 'linear_delta':'True'},
            }

inscaler='minmax' 
outscaler='minmax' 
batch_size = 1
outputsuffix = '_estimate'

respfile = processing_dir + 'y_train.pkl'
covfile = processing_dir + 'x_train.pkl'

testrespfile_path = processing_dir + 'y_test.pkl'
testcovfile_path = processing_dir + 'x_test.pkl'

trbefile = processing_dir + 'b_train.pkl'
tsbefile = processing_dir + 'b_test.pkl'

memory = '2gb'
duration = '2:00:00'
method = 'hetero_SHASH_bspline'
cluster_spec = 'slurm'

In [ ]:
### Running NM

ptk.normative_parallel.execute_nm(processing_dir, python_path,
               'NM', covfile, respfile, batch_size, memory, duration, alg='hbr', 
               log_path=log_path, binary=True, testcovfile_path=testcovfile_path, 
               testrespfile_path=testrespfile_path,trbefile=trbefile, tsbefile=tsbefile, 
               model_type=hbr_configs[method]['model_type'], likelihood=hbr_configs[method]['likelihood'],  
               linear_sigma=hbr_configs[method]['linear_sigma'], random_slope_mu=hbr_configs[method]['random_slope_mu'],
               linear_epsilon=hbr_configs[method]['linear_epsilon'], linear_delta=hbr_configs[method]['linear_delta'], 
               savemodel='True', inscaler=inscaler, outscaler=outscaler, outputsuffix=outputsuffix, 
               interactive='auto', cluster_spec=cluster_spec)

In [ ]:
### Plotting ranges
for i in range(biomarker_num):
    plot_nm_range2(processing_dir, ind=i, parallel=True)

In [1]:
### Running Freesurfer on T1 Anatomical data for source reconstruction

from utils.freesurfer import run_parallel_reconall

mri_directory = '/project/meganorm/Data/camcan/CamCAN/cc700/mri/pipeline/release004/BIDS_20190411/anat'
processing_dir = '/home/meganorm-smkia/temp'
freesurfer_path = '/project/meganorm/Software/freesurfer'
results_directory = '/project/meganorm/Data/camcan/CamCAN/cc700/mri/pipeline/release004/BIDS_20190411/derivatives'

run_parallel_reconall(mri_directory, results_directory, processing_dir, freesurfer_path)


Submitting job for subject: sub-CC212153
Submitting job for subject: sub-CC321428
Submitting job for subject: sub-CC420180
Submitting job for subject: sub-CC720358
Submitting job for subject: sub-CC710494
Submitting job for subject: sub-CC410129
Submitting job for subject: sub-CC320661
Submitting job for subject: sub-CC620090
Submitting job for subject: sub-CC510321
Submitting job for subject: sub-CC510050
Submitting job for subject: sub-CC121685
Submitting job for subject: sub-CC310160
Submitting job for subject: sub-CC121194
Submitting job for subject: sub-CC510086
Submitting job for subject: sub-CC710548
Submitting job for subject: sub-CC510043
Submitting job for subject: sub-CC310473
Submitting job for subject: sub-CC112141
Submitting job for subject: sub-CC621199
Submitting job for subject: sub-CC321595
Submitting job for subject: sub-CC420776
Submitting job for subject: sub-CC221038
Submitting job for subject: sub-CC110101
Submitting job for subject: sub-CC520287
Submitting job f